In [46]:
#classification and regression tree (CART) using the bank note dataset
from random import seed
from random import randrange
from csv import reader

In [47]:
#loading the csv file 
def load_csv(filename):
    file = open(filename, 'rt')
    lines = reader(file)
    dataset = list(lines)
    return dataset

In [48]:
#converting stringfs to float values
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

In [49]:
#splitting the dataset into k-folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range (n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [50]:
#calculating the accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range (len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100

In [51]:
#evaluating the algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

In [52]:
#splitting the dataset based on the attribute and attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

In [53]:
#calculating the gini index for the split dataset
def gini_index(groups, classes):
    n_instances = float(sum(len(group) for group in groups))
    #summing the weighed gini index for each group\
    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
    score = 0.0
    #score the group based on the score in each class
    for class_val in classes:
        p = [row[-1] for row in group].count(class_val) / size
        score += p * p
        #weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini

In [54]:
#selecting the best split point for the dataset
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0]) -1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index' : b_index, 'value' : b_value, 'groups' : b_groups}

In [55]:
#creating a terminal node
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key = outcomes.count)

In [56]:
#creating child split for a node
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    #check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    #check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    #process the left child 
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth +1)
    #prcoessing the right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth +1)

In [57]:
#building the decision tree
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

In [58]:
#making predictions with the decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [61]:
#The classification and Regression Tree Algorithm
def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return (predictions)

In [60]:
#testing the CART on the bank note dataset
seed(1)
filename = 'G:\\data\\ipynb\\bank_note.csv'
dataset = load_csv(filename)

PermissionError: [Errno 13] Permission denied: 'G:\\data\\ipynb\\bank_note.csv'

In [33]:
#converting the string attributes to integeers
for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)
    

In [40]:
#evaluating the algorithm
n_folds = 10
max_depth = 10
min_size = 10
scores = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores) / float(len(scores))))

Scores: [75.18248175182481, 82.48175182481752, 78.1021897810219, 75.91240875912408, 70.8029197080292, 80.2919708029197, 82.48175182481752, 81.02189781021897, 79.56204379562044, 75.91240875912408]
Mean Accuracy: 78.175%
